In [1]:
import osmnx as ox
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import json
print("loaded")
colors = ['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown', 'grey', 'lightblue', 'cyan', 'magenta', 'red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown', 'grey', 'lightblue', 'cyan', 'magenta']
arrondissements = [

    'Ahuntsic-Cartierville',
    'Anjou',
    'Côte-des-Neiges–Notre-Dame-de-Grâce',
    'Lachine',
    'LaSalle',
    'Le Plateau-Mont-Royal',
    'Le Sud-Ouest',
    'L\'Île-Bizard–Sainte-Geneviève',
    'Mercier–Hochelaga-Maisonneuve',
    'Montréal-Nord',
    'Outremont',
    'Pierrefonds-Roxboro',
    'Rivière-des-Prairies–Pointe-aux-Trembles',
    'Rosemont–La Petite-Patrie',
    'Saint-Laurent',
    'Saint-Léonard',
    'Verdun',
    'Ville-Marie',
    'Villeray–Saint-Michel–Parc-Extension'
]

loaded


In [2]:
def get_graph(name):
    G = ox.graph_from_place(name, network_type='drive')
    G = ox.project_graph(G)
    return G

In [3]:
# you can call the getEulerienGraph with the name of the sector
# it will give you the eulerian path of this sector

In [4]:
def read_json(filename):
    with open(filename, "r") as file:
        data = json.loads(file.read())
    return data

def transform_to_tuples(list_of_lists):
    return [tuple(sub_list) for sub_list in list_of_lists]


def getEulerienGraph(name):
    i = arrondissements.index(name)
    print(arrondissements[i])
    chemin = read_json("../drone_scouting/paths.txt")
    return (transform_to_tuples(chemin[i]))
    


In [5]:

paths = getEulerienGraph(arrondissements[0])
print("there is " + str(len(paths)) + " chemins")

Ahuntsic-Cartierville
there is 3277 chemins


In [23]:
# si le chemin n'existe pas dans le circuit non eulerien, on cherche le chemin entre les deux noeuds dans le graphe non eulerien

In [11]:
from collections import deque

def bfs(G, start, end):
    queue = [(start, [start])]
    while queue:
        (node, path) = queue.pop(0)
        for next_node in G.neighbors(node):
            if next_node == end:
                return path + [next_node]
            else:
                queue.append((next_node, path + [next_node]))
    return None



In [7]:
def is_in_graph(G, x, y):
    for (a, b, w) in G:
        if (x == a and y == b):
            return True
    return False

def replace_nodes(G, path):
    i = 0
    print("start while")
    while i < len(path) - 1:
        a = path[i]
        b = path[i + 1]
        if (not is_in_graph(G, a, b)):
            print("dijkstra iter " + str(i))
            p, d = dijkstra(G, a, b, True) # considering graph directed
            if (d == 0): # if there is no path, considering graph non directed
                p, d = dijkstra(G, a, b, False)
            path.pop(i)
            path.pop(i)
            for e in p:
                path.insert(i, e)
                i += 1
            if (d != 0):
                i -= 1
        else:
            i += 1
    print("end while")
    return path

In [8]:
g = get_graph("Ahuntsic-Cartierville" + ", Montreal, Canada")

In [7]:
def dijkstra(G, start, end, is_directed):
    # conversions
    graph = {}
    # calculate total number of vertices
    total = 0
    for (a, b, w) in G:
        if a > total:
            total = a
        if b > total:
            total = b
    for i in range(total + 1):
        graph[i] = {}
    for (a, b, w) in G:
        graph[a][b] = w
        if not is_directed: # if the graph is not directed add the same edge but in reverse direction
            graph[b][a] = w
    
    # empty dictionary to hold distances
    distances = {} 
    # list of vertices in path to current vertex
    predecessors = {} 
    
    # get all the vertices that need to be assessed
    to_assess = graph.keys() 

    # set all initial distances to infinity
    #  and no predecessor for any node
    for node in graph:
        distances[node] = float('inf')
        predecessors[node] = None
    
    # set the initial collection of 
    # permanently labelled nodes to be empty
    sp_set = []

    # set the distance from the start vertex to be 0
    distances[start] = 0
    
    # as long as there are still vertices to assess:
    while len(sp_set) < len(to_assess):

        # chop out any vertices with a permanent label
        still_in = {node: distances[node]\
                    for node in [node for node in\
                    to_assess if node not in sp_set]}

        # find the closest node to the current vertex
        closest = min(still_in, key = distances.get)

        # and add it to the permanently labelled vertices
        sp_set.append(closest)
        
        # then for all the neighbours of 
        # the closest vertex (that was just added to
        # the permanent set)
        for node in graph[closest]:
            # if a shorter path to that node can be found
            if distances[node] > distances[closest] +\
                       graph[closest][node]:

                # update the distance with 
                # that shorter distance
                distances[node] = distances[closest] +\
                       graph[closest][node]

                # set the predecessor for that node
                predecessors[node] = closest
                
    # once the loop is complete the final 
    # path needs to be calculated - this can
    # be done by backtracking through the predecessors
    path = [end]
    while start not in path:
        if (predecessors[path[-1]] == None): # it means that path doesnt exist
            return None, 0
        path.append(predecessors[path[-1]])
    
    # return the path in order start -> end, and it's cost
    return path[::-1], distances[end]


In [9]:

L = []
a,b = paths[0]
L.append(a)

for (a,b) in paths:
    L.append(b)


In [ ]:

true_path = replace_nodes(g.edges, L)

print("The true path is : ", true_path)

start while
dijkstra iter 0


In [12]:
G = [(0, 3, 7), (0, 2, 3), (1, 2, 1), (1, 7, 2), (2, 5, 2), (2, 4, 3), (2, 3, 6), (3, 8, 9), (3, 0, 2), (4, 5, 2), (4, 8, 3), (5, 4, 2), (5, 1, 4), (6, 8, 2), (6, 5, 6), (6, 7, 4), (7, 6, 4), (7, 9, 4), (8, 4, 3), (8, 3, 3), (9, 6, 4), (9, 11, 1), (10, 11, 1)]
eulerian_path = [10, 11, 9, 11, 6, 7, 9, 6, 5, 1, 7, 6, 8, 3, 0, 2, 3, 0, 3, 8, 4, 2, 4, 5, 1, 2, 5, 4, 8]
true_path = replace_nodes(G, eulerian_path)
# Print the final real path
print("The true path is : ", true_path)

start while
dijkstra iter 1


AttributeError: 'list' object has no attribute 'neighbors'